# Signal Matching Analysis for SfN Poster

In [ ]:
from time import gmtime, strftime
import numpy as np
import webknossos as wk
import pandas as pd
%matplotlib notebook
import matplotlib.pyplot as plt
import skimage
from at_synapse_detection import SynapseDetection as syn
from PIL import Image, ImageSequence
from skimage import measure
from tqdm import tqdm
from scipy import ndimage
import imageio
from skimage import io
import skimage.io as skio

## Synapse detection

In [ ]:
psd_vol = skio.imread('data/F002-PSD95-4th.tif', plugin='tifffile')
psd_data = np.transpose(psd_vol, (1, 2, 0))

syn_vol = skio.imread('data/F002-Syn12-2nd.tif', plugin='tifffile')
synapsin_data = np.transpose(syn_vol, (1, 2, 0))

synapsin_data = np.double(synapsin_data)
psd_data = np.double(psd_data)

# select only slices 5-48
synapsin_data = synapsin_data[:, :, 4:]
psd_data = psd_data[:, :, 4:]


# Create a dictionary object to hold the image data 
synaptic_volumes = {'presynaptic': [synapsin_data], 'postsynaptic': [psd_data]}

# Specify the minimum number of slices each blob should span 
min_num_of_slices = 1

# Create query dictionary object 
query = {'preIF': ['synapsin_data',], 'preIF_z': [1],
         'postIF': ['psd_data'], 'postIF_z': [2],
         'punctumSize': 2}

result_vol = syn.getSynapseDetections(synaptic_volumes, query)
label_vol, counts = measure.label(result_vol>0.8, connectivity=3, return_num=True)    
result_props = measure.regionprops(label_vol)

In [ ]:
for n in range(0, label_vol.shape[2]): 
    imageio.imwrite('detections/detection' + str(n).zfill(2) + '.tiff', label_vol[:, :, n])

In [ ]:
arealist = [] 
slicelist = [] 

for n in range(0, len(result_props)): 
    slicelist.append(result_props[n].bbox[5]-result_props[n].bbox[2])
    arealist.append(result_props[n].area)

In [ ]:
detection_list = list(range(1, len(result_props)+1))

In [ ]:
df = pd.DataFrame(columns=['Detection_label','Detection_slices', 'Detection_pixels'])
df['Detection_label'] = detection_list

df['Detection_slices'] = slicelist
df['Detection_pixels'] = arealist
# df.to_csv('detection_information_oct25_2022.csv')

In [ ]:
filelist = ['F002-GABA-4th.tif',\
'F002-Geph-4th.tif',\
'F002-GluA1-1st.tif',\
'F002-GluA2-2nd.tif',\
'F002-GluA3-2nd.tif',\
'F002-GluA4-3rd.tif',\
'F002-GluN1-1st.tif',\
'F002-GluN2-3rd.tif',\
'F002-MBP64-3rd.tif',\
'F002-PSD95-4th.tif',\
'F002-Syn12-2nd.tif',\
'F002-VGluT1-1st.tif']

## Match PSD signal 

In [ ]:
## Reload PSD 
psd_vol = skio.imread('data/F002-PSD95-4th.tif', plugin='tifffile')
psd_data = np.transpose(psd_vol, (1, 2, 0))
psd_data = np.double(psd_data)

# select only slices 5-48
psd_data = psd_data[:, :, 4:]

# Create a dictionary object to hold the image data 
synaptic_volumes = {'presynaptic': [], 'postsynaptic': [psd_data]}

# Specify the minimum number of slices each blob should span 
min_num_of_slices = 1

# Create query dictionary object 
query = {'preIF': [], 'preIF_z': [1],
         'postIF': ['psd_data'], 'postIF_z': [1],
         'punctumSize': 2}

psd_probability_map = syn.getSynapseDetections(synaptic_volumes, query)
    
# np.savez('PSD1_SYN1.npz', result_vol)
psd_label_vol, counts = measure.label(psd_probability_map>0.8, connectivity=3, return_num=True)
psd_props = measure.regionprops(psd_label_vol)

In [ ]:
for n in range(0, label_vol.shape[2]): 
    imageio.imwrite('psd_label/detection' + str(n).zfill(2) + '.tiff', psd_label_vol[:, :, n])

In [ ]:
for annoitr in range(0, len(result_props)): 
    foo = result_props[annoitr]
    psdlabellist = []
    for n in range(0, len(foo.coords)): 
        psdlabellist.append(psd_label_vol[foo.coords[n][0], foo.coords[n][1], foo.coords[n][2]]) 
    
    
    if len(np.unique(psdlabellist)) > 1: 
        print(annoitr, len(np.unique(psdlabellist))) 
    if np.unique(psdlabellist)[0] == 0: 
        print("zero", annoitr)

In [ ]:
psd_vol = skio.imread('normalized_data/F002-PSD95-4thP.tif', plugin='tifffile')
psd_data = np.transpose(psd_vol, (1, 2, 0))

psd_data = np.double(psd_data)

# select only slices 5-48
psd_data = psd_data[:, :, 4:]

PSD_label_list = [] 
intensity_sum_list = [] 
intensity_avg_list = []
number_of_slices_list = [] 
number_of_pixels_list = [] 


for n_detection in tqdm(range(0, len(result_props))): 
    
    detection_object = result_props[n_detection]
    psdlabel_number = psd_label_vol[detection_object.coords[0][0],\
                                    detection_object.coords[0][1],\
                                    detection_object.coords[0][2]]
     
    psd_object = psd_props[psdlabel_number-1]
    PSD_label_list.append(psdlabel_number)              
    output_list = [] 
    
    mask = psd_label_vol==psdlabel_number
    masked_vol = psd_data*mask
    summed_intensity = np.sum(masked_vol)
    avg_intensity = summed_intensity/len(psd_object.coords)

    
    if len(psd_object.coords) > 500: 
        intensity_sum_list.append(np.nan)
        intensity_avg_list.append(np.nan)
        number_of_slices_list.append(np.nan)
        number_of_pixels_list.append(np.nan)
    else: 
        intensity_sum_list.append(summed_intensity)
        intensity_avg_list.append(avg_intensity)
        number_of_slices_list.append(psd_object.bbox[5] - psd_object.bbox[2] + 1)
        number_of_pixels_list.append(len(psd_object.coords))

In [ ]:
df['PSD_label'] = PSD_label_list

df['PSD_intensity_sum'] = intensity_sum_list
df['PSD_intensity_avg'] = intensity_avg_list
df['PSD_slices'] = number_of_slices_list
df['PSD_pixels'] = number_of_pixels_list

In [ ]:
len(number_of_slices_list) 

In [ ]:
np.sum(pd.isnull(df['PSD_intensity_avg'])) 

## SYNAPSIN

In [ ]:
## Reload Synapsin 
syn_vol = skio.imread('data/F002-Syn12-2nd.tif', plugin='tifffile')
syn_vol = np.transpose(syn_vol, (1, 2, 0))
syn_vol = np.double(syn_vol)

# select only slices 5-48
syn_vol = syn_vol[:, :, 4:]

# Create a dictionary object to hold the image data 
synaptic_volumes = {'presynaptic': [], 'postsynaptic': [syn_vol]}

# Specify the minimum number of slices each blob should span 
min_num_of_slices = 1

# Create query dictionary object 
query = {'preIF': [], 'preIF_z': [1],
         'postIF': ['syn_vol'], 'postIF_z': [1],
         'punctumSize': 2}

syn_probability_map = syn.getSynapseDetections(synaptic_volumes, query)
    
# np.savez('PSD1_SYN1.npz', result_vol)
syn_label_vol, counts = measure.label(syn_probability_map>0.8, connectivity=3, return_num=True)
syn_props = measure.regionprops(syn_label_vol)

In [ ]:
plt.figure()
plt.imshow(syn_vol[60:110, 375:440, 15])
plt.colorbar()

In [ ]:
plt.figure()
plt.imshow(syn_probability_map[60:110, 375:440, 15])
plt.colorbar()

In [ ]:
for n in range(0, label_vol.shape[2]): 
    imageio.imwrite('syn_label/detection' + str(n).zfill(2) + '.tiff', syn_label_vol[:, :, n])

In [ ]:
matched_synapsin_list = [] 

for annoitr in range(0, len(result_props)): 
    biggest_label = 0 
    result_object = result_props[annoitr]
    synlabellist = []

    if result_object.bbox[1]<=3: 
        mincol = 0
    else: 
        mincol = result_object.bbox[1] - 3
    if result_object.bbox[4]>=syn_label_vol.shape[1]-3: 
        maxcol = result_object.bbox[4]
    else: 
        maxcol = result_object.bbox[4] + 3

    if result_object.bbox[0]<=3: 
        minrow = 0
    else: 
        minrow = result_object.bbox[0] - 3
    if result_object.bbox[3]>=syn_label_vol.shape[0]-3: 
        maxrow = result_object.bbox[3]
    else: 
        maxrow = result_object.bbox[3] + 3

    if result_object.bbox[2]==0: 
        minslice = 0
    else: 
        minslice = result_object.bbox[2] - 1
    if result_object.bbox[5]==syn_label_vol.shape[2]-1: 
        maxslice = result_object.bbox[5]
    else: 
        maxslice = result_object.bbox[5] + 1




    output = syn_label_vol[minrow:maxrow,\
                           mincol:maxcol,\
                           minslice:maxslice]
    unique_vals, unique_counts = np.unique(output, return_counts=True)
    if unique_vals[0]==0 and len(unique_vals)>0: 

        if len(unique_vals) > 2: 
            biggest_label_ind = np.argmax(unique_counts[1:])
            biggest_label = unique_vals[biggest_label_ind+1]
            
        elif len(unique_vals) == 2: 
            biggest_label = unique_vals[1]
        
    else: 
        biggest_label = unique_vals[0]
        print(unique_vals[0])
       
    matched_synapsin_list.append(biggest_label)

In [ ]:
len(matched_synapsin_list)

In [ ]:
len(np.where(np.array(matched_synapsin_list) == 0) [0]) 

In [ ]:
## Reload Synapsin 
syn_vol = skio.imread('normalized_data/F002-Syn12-2ndP.tif', plugin='tifffile')
syn_vol = np.transpose(syn_vol, (1, 2, 0))
syn_vol = np.double(syn_vol)

# select only slices 5-48
syn_vol = syn_vol[:, :, 4:]

intensity_sum_list = [] 
intensity_avg_list = []
number_of_slices_list = [] 
number_of_pixels_list = [] 

for syn_label in tqdm(matched_synapsin_list): 
    
    if syn_label == 0: 
        intensity_sum_list.append(0)
        intensity_avg_list.append(0)
        number_of_slices_list.append(0)
        number_of_pixels_list.append(0)
    
    else: 
        syn_object = syn_props[syn_label-1]

        output_list = [] 

        mask = syn_label_vol==syn_label
        masked_vol = syn_vol*mask
        summed_intensity = np.sum(masked_vol)
        avg_intensity = summed_intensity/len(syn_object.coords)

        
        if len(syn_object.coords) > 4500: 
            intensity_sum_list.append(np.nan)
            intensity_avg_list.append(np.nan)
            number_of_slices_list.append(np.nan)
            number_of_pixels_list.append(np.nan)
        else: 
            intensity_sum_list.append(summed_intensity)
            intensity_avg_list.append(avg_intensity)
            number_of_slices_list.append(syn_object.bbox[5] - syn_object.bbox[2] + 1)
            number_of_pixels_list.append(len(syn_object.coords))

In [ ]:

df['SYN_label'] = matched_synapsin_list
df['SYN_intensity_sum'] = intensity_sum_list
df['SYN_intensity_avg'] = intensity_avg_list
df['SYN_slices'] = number_of_slices_list
df['SYN_pixels'] = number_of_pixels_list

## Find postsynaptic overlap with PSD

In [ ]:
def loaddata(fn): 
    syn_vol = skio.imread(fn, plugin='tifffile')
    syn_vol = np.transpose(syn_vol, (1, 2, 0))
    syn_vol = np.double(syn_vol)

    # select only slices 5-48
    syn_vol = syn_vol[:, :, 4:]
    
    return syn_vol

In [ ]:
def getSingleSliceDetections(fn): 
    # fn - 'data/F002-Syn12-2nd.tif'
    syn_vol = skio.imread(fn, plugin='tifffile')
    syn_vol = np.transpose(syn_vol, (1, 2, 0))
    syn_vol = np.double(syn_vol)

    # select only slices 5-48
    syn_vol = syn_vol[:, :, 4:]

    # Create a dictionary object to hold the image data 
    synaptic_volumes = {'presynaptic': [], 'postsynaptic': [syn_vol]}

    # Specify the minimum number of slices each blob should span 
    min_num_of_slices = 1

    # Create query dictionary object 
    query = {'preIF': [], 'preIF_z': [1],
             'postIF': ['syn_vol'], 'postIF_z': [1],
             'punctumSize': 2}

    syn_probability_map = syn.getSynapseDetections(synaptic_volumes, query)

    # np.savez('PSD1_SYN1.npz', result_vol)
    syn_label_vol, counts = measure.label(syn_probability_map>0.8, connectivity=3, return_num=True)
    syn_props = measure.regionprops(syn_label_vol)
    
    return syn_label_vol, syn_props

In [ ]:
receptor_filelist = \
['data/F002-GluA1-1st.tif',\
'data/F002-GluA2-2nd.tif',\
'data/F002-GluA3-2nd.tif',\
'data/F002-GluA4-3rd.tif',\
'data/F002-GluN1-1st.tif',\
'data/F002-GluN2-3rd.tif']

In [ ]:
normalized_receptor_filelist = \
['normalized_data/F002-GluA1-1stP.tif',\
'normalized_data/F002-GluA2-2ndP.tif',\
'normalized_data/F002-GluA3-2ndP.tif',\
'normalized_data/F002-GluA4-3rdP.tif',\
'normalized_data/F002-GluN1-1stP.tif',\
'normalized_data/F002-GluN2-3rdP.tif']

In [ ]:
# for each PSD blob, find blob somewhere else
receptor_matched_list = []

for n_fn, fn in enumerate(receptor_filelist): 
    print(fn)
    intensity_sum_list = [] 
    intensity_avg_list = []
    number_of_slices_list = [] 
    number_of_pixels_list = [] 

    matched_receptor_list = [] 

    for n in tqdm(range(0, len(PSD_label_list))): 
        psdlabel_number = PSD_label_list[n]
        psd_object = psd_props[psdlabel_number-1]
        psd_mask = psd_label_vol==psdlabel_number
        
        data = loaddata(normalized_receptor_filelist[n_fn])
        
        output = data*psd_mask
            
        summed_intensity = np.sum(output)

        if summed_intensity==0: 
            intensity_sum_list.append(0)
            intensity_avg_list.append(0)
            number_of_slices_list.append(0)
            number_of_pixels_list.append(0)
        else: 
            data_label_vol, counts = measure.label(output>0, connectivity=3, return_num=True)
            data_props = measure.regionprops(data_label_vol)
            data_object = data_props[0]
            
            avg_intensity = summed_intensity/len(data_object.coords)

            intensity_sum_list.append(summed_intensity)
            intensity_avg_list.append(avg_intensity)
            number_of_slices_list.append(data_object.bbox[5] - data_object.bbox[2] + 1)
            number_of_pixels_list.append(len(data_object.coords))

                
    channel_name = fn.split('-')[1]
    ch_label = channel_name + '_label'
    intensity_sum = channel_name + '_intensity_sum'
    intensity_avg = channel_name + '_intensity_avg'
    ch_slices = channel_name + '_slices'
    ch_pixels = channel_name + '_pixels'
    
    df[intensity_sum] = intensity_sum_list
    df[intensity_avg] = intensity_avg_list
    df[ch_slices] = number_of_slices_list
    df[ch_pixels] = number_of_pixels_list

In [ ]:
transmitter_filelist = ['normalized_data/F002-VGluT1-1stP.tif']

In [ ]:
len(matched_synapsin_list) 

In [ ]:
# for each synapsin blob, find blob somewhere else

for n_fn, fn in enumerate(transmitter_filelist): 
    print(fn)
    intensity_sum_list = [] 
    intensity_avg_list = []
    number_of_slices_list = [] 
    number_of_pixels_list = [] 

    for n in tqdm(range(0, len(matched_synapsin_list))): 
        synlabel_number = matched_synapsin_list[n]
        syn_object = syn_props[synlabel_number-1]
        syn_mask = syn_label_vol==synlabel_number
        
        data = loaddata(transmitter_filelist[n_fn])
        
        output = data*syn_mask
        
        summed_intensity = np.sum(output)

        if summed_intensity==0: 
            intensity_sum_list.append(0)
            intensity_avg_list.append(0)
            number_of_slices_list.append(0)
            number_of_pixels_list.append(0)
        else: 
            data_label_vol, counts = measure.label(output>0, connectivity=3, return_num=True)
            data_props = measure.regionprops(data_label_vol)
            data_object = data_props[0]
            
            avg_intensity = summed_intensity/len(data_object.coords)

            intensity_sum_list.append(summed_intensity)
            intensity_avg_list.append(avg_intensity)
            number_of_slices_list.append(data_object.bbox[5] - data_object.bbox[2] + 1)
            number_of_pixels_list.append(len(data_object.coords))

                
    channel_name = fn.split('-')[1]
    ch_label = channel_name + '_label'
    intensity_sum = channel_name + '_intensity_sum'
    intensity_avg = channel_name + '_intensity_avg'
    ch_slices = channel_name + '_slices'
    ch_pixels = channel_name + '_pixels'
    
    df[intensity_sum] = intensity_sum_list
    df[intensity_avg] = intensity_avg_list
    df[ch_slices] = number_of_slices_list
    df[ch_pixels] = number_of_pixels_list
    

In [ ]:
df

In [ ]:
df.to_csv('results_nov8_2022.csv')